## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-02-23-46-00 +0000,wsj,Gold Falls on Reports of China’s Finance Minis...,https://www.wsj.com/finance/commodities-future...
1,2025-11-02-23-42-30 +0000,nypost,Fake Yale student scandal raises alarms over a...,https://nypost.com/2025/11/02/us-news/fake-yal...
2,2025-11-02-23-41-55 +0000,cbc,"Mississippi mom feared for children's safety, ...",https://www.cbc.ca/news/world/escaped-monkey-s...
3,2025-11-02-23-41-52 +0000,nypost,Cross-dressing porch pirate caught swiping pac...,https://nypost.com/2025/11/02/us-news/cross-dr...
4,2025-11-02-23-40-21 +0000,latimes,Not registered to vote yet? It's not too late ...,https://www.latimes.com/politics/story/2025-11...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2387/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
151,trump,22
390,train,10
69,mamdani,9
233,new,9
431,military,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
59,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...,63
88,2025-11-02-17-00-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,57
177,2025-11-02-00-19-33 +0000,wapo,Trump threatens potential military action in N...,https://www.washingtonpost.com/world/2025/11/0...,57
85,2025-11-02-17-12-06 +0000,bbc,Trump tells military to prepare for 'action' a...,https://www.bbc.com/news/articles/cev18jy21w7o...,53
108,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
59,63,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...
108,51,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...
27,37,2025-11-02-21-45-19 +0000,nypost,Queens Democrat Party snubs own Assemblyman Zo...,https://nypost.com/2025/11/02/us-news/queens-d...
141,36,2025-11-02-10-00-00 +0000,wsj,"Republican Jack Ciattarelli, locked in a tight...",https://www.wsj.com/politics/elections/new-jer...
88,35,2025-11-02-17-00-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...
72,34,2025-11-02-18-44-31 +0000,nypost,"Maniac chops up mom, stores bagged remains in ...",https://nypost.com/2025/11/02/us-news/maniac-c...
95,31,2025-11-02-16-11-31 +0000,bbc,"'No help, no food, no water': Hurricane-hit Ja...",https://www.bbc.com/news/articles/cpv11w8ez10o...
60,29,2025-11-02-19-59-19 +0000,nypost,Ex-Prince Andrew could be dealt another humili...,https://nypost.com/2025/11/02/world-news/ex-pr...
74,28,2025-11-02-18-36-44 +0000,latimes,Photos: Los Angeles celebrates Dodgers' first ...,https://www.latimes.com/california/story/2025-...
67,27,2025-11-02-19-17-09 +0000,nypost,Gavin Newsom claims he ‘never had’ concerns ab...,https://nypost.com/2025/11/02/us-news/newsom-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
